In [1]:
from ultralytics import YOLO
import os

In [2]:
data_path = os.path.join(os.getcwd(), 'datasets', 'AblationDataset', 'flow', 'flow.yaml')
test_data_path = os.path.join(os.getcwd(), 'datasets', 'TestDataset', 'flowtest', 'flowtest.yaml')

In [3]:
model = YOLO('yolo12-simam.yaml')
model.train(
    data=data_path,
    epochs=50,
    batch=12,
    imgsz=640,
    name='yolo12-simam', 
)

WARNING  no model scale passed. Assuming scale='s'.
New https://pypi.org/project/ultralytics/8.3.103 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.96  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)
engine\trainer: task=detect, mode=train, model=yolo12-simam.yaml, data=c:\Users\MUHAMMADRAFLYARJASUB\Documents\BojongTrash\ultralytics-floating-trash\datasets\AblationDataset\flow\flow.yaml, epochs=50, time=None, patience=100, batch=12, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=yolo12-simam, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=Tr

train: Scanning C:\Users\MUHAMMADRAFLYARJASUB\Documents\BojongTrash\ultralytics-floating-trash\datasets\AblationDataset\flow\train\labels.cache... 961 images, 0 backgrounds, 0 corrupt: 100%|██████████| 961/961 [00:00<?, ?it/s]
val: Scanning C:\Users\MUHAMMADRAFLYARJASUB\Documents\BojongTrash\ultralytics-floating-trash\datasets\AblationDataset\flow\valid\labels.cache... 239 images, 1 backgrounds, 0 corrupt: 100%|██████████| 239/239 [00:00<?, ?it/s]


Plotting labels to runs\detect\yolo12-simam\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.002, momentum=0.9) with parameter groups 113 weight(decay=0.0), 120 weight(decay=0.00046875), 119 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\yolo12-simam
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50      5.11G      5.365      19.44      3.778          1        640: 100%|██████████| 81/81 [01:19<00:00,  1.02it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.44it/s]

                   all        239        628   4.22e-05    0.00318   2.17e-05   4.34e-06


...

                   all        239        628      0.904      0.721      0.813      0.393



50 epochs completed in 1.152 hours.
Optimizer stripped from runs\detect\yolo12-simam\weights\last.pt, 18.9MB
Optimizer stripped from runs\detect\yolo12-simam\weights\best.pt, 18.9MB

Validating runs\detect\yolo12-simam\weights\best.pt...
Ultralytics 8.3.96  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)
YOLO12-simam summary (fused): 162 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 10/10 [00:04<00:00,  2.32it/s]


                   all        239        628       0.87      0.732      0.812        0.4
Speed: 0.3ms preprocess, 9.1ms inference, 0.0ms loss, 1.9ms postprocess per image
Results saved to runs\detect\yolo12-simam


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000011B1245BB80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

In [4]:
model.val(data=test_data_path)

Ultralytics 8.3.96  Python-3.9.21 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce GTX 1060 6GB, 6144MiB)
YOLO12-simam summary (fused): 162 layers, 9,231,267 parameters, 0 gradients, 21.2 GFLOPs


val: Scanning C:\Users\MUHAMMADRAFLYARJASUB\Documents\BojongTrash\ultralytics-floating-trash\datasets\TestDataset\flowtest\test\labels.cache... 800 images, 1 backgrounds, 0 corrupt: 100%|██████████| 800/800 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 67/67 [00:13<00:00,  4.86it/s]


                   all        800       2022      0.805      0.661      0.727      0.326
Speed: 0.2ms preprocess, 10.1ms inference, 0.0ms loss, 1.3ms postprocess per image
Results saved to runs\detect\yolo12-simam2


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x0000011B12712610>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480